<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">22.418 · Aprenentatge automàtic</p>
<p style="margin: 0; text-align:right;">Grau en Ciència de Dades Aplicada</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Module 2: Naive Bayes example

In this notebook we will implement a Naive Bayes classifier to distinguish between the different classes existent in two different datasets: the [Mushroom data set](https://archive.ics.uci.edu/ml/datasets/Mushroom) and the <a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">Iris flower data set</a>.

We will start with a binary dataset (i.e. the Mushroom data set) and then move to a multi-class data set and learn how scikit-learn is able to automatically deal with multi-class for us.

The Mushroom dataset contains data on which mushrooms are edible and which are poisonous. There are 8124 mushrooms defined in the dataset of which 4208 are edible and 3915 are poisonous. Each of these is characterized by 22 features.

On the other hand, the Iris flower dataset consists of 3 different types of irises' (Setosa, Versicolor, and Virginica) petal and sepal length, stored in a 150x4 numpy ndarray. The rows being the samples and the columns being: Sepal Length, Sepal Width, Petal Length and Petal Width.

First of all, let's see the libraries we will use during our practical assignment:

### [numpy](https://numpy.org)

The NumPy library for Python concentrates on handling extensive multi-dimensional data and the intricate mathematical functions operating on the data. NumPy offers speedy computation and execution of complicated functions working on arrays. Few of the points in favor of NumPy are:

- Support for mathematical and logical operations
- Shape manipulation
- Sorting and Selecting capabilities
- Discrete Fourier transformations
- Basic linear algebra and statistical operations
- Random simulations
- Support for n-dimensional arrays

NumPy works on an object-oriented approach and has tools for integrating C, C++ and Fortran code, and this makes NumPy highly popular amongst the scientific community.

### [pandas](https://pandas.pydata.org)

Pandas is a Python data analysis library and is used primarily for data manipulation and analysis. It comes into play before the dataset is prepared for training. Pandas make working with time series and structured multidimensional data effortless for machine-learning programmers. Some of the great features of Pandas when it comes to handling data are:

- Dataset reshaping and pivoting
- Merging and joining of datasets
- Handling of missing data and data alignment
- Various indexing options such as Hierarchical axis indexing, Fancy indexing
- Data filtration options

Pandas makes use of DataFrames, which is just a technical term for a two-dimensional representation of data by offering programmers with DataFrame objects.

### [scikit-learn](https://scikit-learn.org)

Scikit-learn is a very actively used machine learning library for Python. It includes easy integration with different ML programming libraries like NumPy and Pandas. Scikit-learn comes with the support of various algorithms such as:

- Classification
- Regression
- Clustering
- Dimensionality Reduction
- Model Selection
- Preprocessing

Built around the idea of being easy to use but still be flexible, Scikit-learn is focussed on data modelling and not on other tasks such as loading, handling, manipulation and visualization of data. It is considered sufficient enough to be used as an end-to-end ML, from the research phase to the deployment.

### [matplotlib](https://matplotlib.org)

Simply put, matplotlib is a plotting library. If you’ve ever used MATLAB before, you’ll probably feel very comfortable in the matplotlib environment. When analyzing images, we’ll make use of matplotlib, whether plotting the overall accuracy of search systems or simply viewing the image itself, matplotlib is a great tool to have in your toolbox.

Matplotlib is a comprehensive data visualization library in Python. Some features include:

- creating interactive plots.
- offering flexible customization of the plot.

It is a foundation library supporting seaborn, which is easier to use. But when we want to customize the plots more, matplotlib becomes necessary. It's also useful for quick tests and visualizations.

### [seaborn](https://seaborn.pydata.org)

Seaborn is a popular Python library for making statistical data visualizations. It is based on matplotlib and also integrated with pandas data structures.

Seaborn is especially useful for exploring and understanding data. Some of the functionalities that seaborn offers:

- options for visualizing univariate and bivariate distributions.
- options for visualizing numerical and categorical variables.
- automatic estimation and plotting of linear regression models.
- ability to build complex visualizations such as multi-plot grids.

It comes in handy whenever whe need to explore a dataset with Histograms, Heatmaps, Scatter plots, Barplots, etc.


This is a general introduction of each of the libraries we will use. For more information, feel free to refer to their official webpages.


### Mushrooms data set

Now, as we said, we will start with the Mushroom dataset. To do so, we will first import everything we need:

In [ ]:
# Import packages
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

After loading the libraries, we need to load the data.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data
!cat agaricus-lepiota.data | head 

The first column shows the class: "p" for poisonous, "e" for edible. The rest of columns are the features. Let's load the data:

In [ ]:
mushrooms = pd.read_csv('agaricus-lepiota.data')
mushrooms.head()

Divide between the feature variables and the class: 

In [ ]:
mushrooms_X = mushrooms.drop(['p'], axis=1)
y = mushrooms.values[:, 0]

All the variables are categorical, so we need to convert them to numerical.

In [ ]:
# All the variables are in string format. Convert categorical variables to integer using pandas get_dummies function

for col in mushrooms_X.columns:
  mushrooms_X = pd.get_dummies(mushrooms_X,prefix=[col], columns = [col], drop_first=False)

In [ ]:
mushrooms_X.head()

Now that we have the dataset all with numerical values, we can separate the data and the labels and split in training and testing set.

As you all already know, in order to carry out experiments, we need to split the dataset in two subsets, one for training and another one for testing. We'll do it using scikit-learn `train_test_split` function, as we learnt in the first unit.

In [ ]:
x = mushrooms_X.values[:, 1:]


In [ ]:
# Split train and test
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33, random_state=42)

Let's check the shape of each subset:

In [ ]:
print('xtrain shape: ', xtrain.shape)
print('ytrain shape: ', ytrain.shape)
print('xtest shape: ', xtest.shape)
print('ytest shape: ', ytest.shape)

At this point we have everything we need to start implementing our classifier. In this example we will use the Naive Bayes rule to classify the different samples in our dataset.

`scikit-learn` implements such a classifier in its `GaussianNB` class. It only has two parameters:

```
priors: array-like of shape (n_classes,)
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.

var_smoothing: float, default=1e-9
Portion of the largest variance of all features that is added to variances for calculation stability.
```

For our task, we'll leave them by default.

Let's create the classifier:

In [ ]:
# Init the Gaussian Classifier
model = GaussianNB()

Now, we can use the `train` method to fit the classifier to our train set:

In [ ]:
# Train the model 
model.fit(xtrain, ytrain)

And once it's trained, use the model to predict the samples in the test set:

In [ ]:
# Predict Output 
preds = model.predict(xtest)

Let's see the result. We'll compute the confusion matrix, which contains information about true positives/negatives and false positives/negatives.

In [ ]:
# Plot Confusion Matrix
mat = confusion_matrix(ytest, preds)
names = np.unique(preds)
sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=names, yticklabels=names)
plt.xlabel('Truth')
plt.ylabel('Predicted')

We can also compute the most common classification metrics with the help of `sklearn.metrics.classification_report`. These metrics include the precission, recall, f1-score and support of each class. For more information regarding these metrics, you can refer to the theoretical documents of the course or access the <a hred="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html">documentation</a>.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, preds))

Furthermore, thanks to the probabilistic behaviour of the Naive Bayes classifier, we can easily access the probability of each instance belonging to each lass:

In [ ]:
preds_proba = model.predict_proba(xtest)
print(preds_proba[:10, :])

These probabilities show the probability of each instance to belong to each of the two classes: edible and poisonous.

### Iris flower dataset

For the Iris flower dataset, we'll use scikit-learn's `datasets` class, which gives easy access to common toy datasets used in Machine Learning/Deep Learning.

In this example, we will deal with a multiclass problem automatically thanks to the internal mechanisms of `scikit-learn`, that will deal with it for us. From now on, we will carry out all the examples with the Iris flower dataset.

The data itself is contained in `iris.data`, and the labels are in `iris.target`.

In [ ]:
# Import data
iris = datasets.load_iris()
x = iris.data
y = iris.target

As you all already know, in order to carry out experiments, we need to split the dataset in two subsets, one for training and another one for testing. We'll do it using scikit-learn `train_test_split` function, as we learnt in the first unit.

In [ ]:
# Split train and test
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33, random_state=42)

Let's check the shape of each subset:

In [ ]:
print('xtrain shape: ', xtrain.shape)
print('ytrain shape: ', xtrain.shape)
print('xtest shape: ', xtest.shape)
print('ytest shape: ', xtest.shape)

At this point we have everything we need to start implementing our classifier. In this example we will use the Naive Bayes rule to classify the different samples in our dataset.

`scikit-learn` implements such a classifier in its `GaussianNB` class. It only has two parameters:

```
priors: array-like of shape (n_classes,)
Prior probabilities of the classes. If specified the priors are not adjusted according to the data.

var_smoothing: float, default=1e-9
Portion of the largest variance of all features that is added to variances for calculation stability.
```

For our task, we'll leave them by default.

Let's create the classifier:

In [ ]:
# Init the Gaussian Classifier
model = GaussianNB()

Now, we can use the `train` method to fit the classifier to our train set:

In [ ]:
# Train the model 
model.fit(xtrain, ytrain)

And once it's trained, use the model to predict the samples in the test set:

In [ ]:
# Predict Output 
preds = model.predict(xtest)

Let's see the result. We'll compute the confusion matrix, which contains information about true positives/negatives and false positives/negatives.

In [ ]:
# Plot Confusion Matrix
mat = confusion_matrix(preds, ytest)
names = np.unique(preds)
sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=names, yticklabels=names)
plt.xlabel('Truth')
plt.ylabel('Predicted')

As you can see, the model behaves almost perfectly, with only 2 miss-classifications.

We can also compute the most common classification metrics with the help of `sklearn.metrics.classification_report`. These metrics include the precission, recall, f1-score and support of each class. For more information regarding these metrics, you can refer to the theoretical documents of the course or access the <a hred="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html">documentation</a>.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, preds))

Furthermore, thanks to the probabilistic behaviour of the Naive Bayes classifier, we can easily access the probability of each instance belonging to each lass:

In [ ]:
preds_proba = model.predict_proba(xtest)
print(preds_proba[:10, :])

These probabilities show the probability of each instance to belong to each of the three classes: setosa, versicolor and virginica.

Finally, we can also graphically show the decision boundaries for the 3 classes. However, since we will visualize them in 2 dimensions, we'll need to re-train the algorithm with just 2 of the 4 available variables. For this example, we'll take the first 2, but feel free to try with any other combination!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

X = iris.data[:, :2] # to visualize in 2-dim, we can only take 2 variables
h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue'])

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
# Train the model 
model.fit(xtrain[:, :2], ytrain) # to visualize in 2-dim, we can only take 2 variables
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
plt.title("3-Class classification")
plt.axis('tight')

Another possible option would be to apply a PCA to the Iris data to reduce the number of dimensions to just 2 and then use the transformed data to visualise the decision boundaries. You are welcome to try it!